In [5]:
import pandas as pd
import os
import random
import re

In [ ]:
def create_training_sample(individual_pathway, all_nodes, union_ppi, ind_pathway_name):
    """
    create the training samples from a processed single pathway
    """
    sample_size = max(1, len(all_nodes) // 4)  # TODO: change this to multiplication
    # randomly sample nodes
    random_sample = random.sample(all_nodes, sample_size)
    
    # Filter the edges that contain the sampled nodes
    mask = individual_pathway[0].isin(random_sample) & individual_pathway[2].isin(random_sample)
    matching_rows = individual_pathway[mask]
    
    # Generate the training sample
    training_sample = union_ppi.copy()
    training_sample["label"] = training_sample[[0, 1]].apply(tuple, axis=1).isin(matching_rows.apply(tuple, axis=1)).astype(int)
    
    if 1 in training_sample['label'].tolist():
        print("True labels in PPI - found")
    else:
        print("No True Label found in PPI")
        
    # save the training sample
    filename = ind_pathway_name + str(random.randint(1000, 9999)) + '.csv'
    training_sample.to_csv(f'training-samples/{filename}', index=False, sep='\t')

In [ ]:
def process_individual_pathway(file_path, union_ppi, id_map):
    """
    Process a single pathway file and generate training samples
    
    Parameters:
    file_path (str): Path to the pathway file
    union_ppi (pd.DataFrame): The union PPI dataset
    id_map (pd.DataFrame): Mapping between gene names and entry names
    """
    # extract filename without extension for naming training samples
    path_name = os.path.splitext(os.path.basename(file_path))[0]
    path_name = re.sub(r'\W+', '_', path_name).strip('_').lower() + '_train_'
    print(path_name)
    
    # read + preprocess the pathway file
    individual_pathway = pd.read_csv(file_path, sep='\t', header=None)
    # remove the reaction column
    individual_pathway.drop(columns=[1], inplace=True)
    
    # create mapping dictionary from id map
    mapping_dict = dict(zip(id_map["gene_name"], id_map["entry_name"]))
    # replace gene names w entry names
    individual_pathway[0] = individual_pathway[0].replace(mapping_dict)
    individual_pathway[2] = individual_pathway[2].replace(mapping_dict)
    
    # get all unique nodes in the individual pathway
    all_nodes = list(set(individual_pathway[0].tolist() + individual_pathway[2].tolist()))
    
    print(f"\nProcessing pathway: {path_name}")
    print(f"Total nodes in pathway: {len(all_nodes)}")
    
    # Generate 10 training samples for each pathway
    for i in range(10):
        create_training_sample(individual_pathway, all_nodes, union_ppi, path_name)

In [ ]:
def process_all_pathways(folder_path, union_ppi, id_map):
    """
    Process all pathway files in the specified folder
    
    Parameters:
    folder_path (str): Path to the folder containing pathway files
    union_ppi (pd.DataFrame): The union PPI dataset
    id_map (pd.DataFrame): Mapping between gene names and entry names
    """

    os.makedirs('training-samples', exist_ok=True)
    # get all .txt files in the folder
    pathway_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
    print(f"Found {len(pathway_files)} pathway files to process")
    
    # Process each pathway file
    for file_name in pathway_files:
        file_path = os.path.join(folder_path, file_name)
        try:
            process_individual_pathway(file_path, union_ppi, id_map)
        except Exception as e:
            print(f"Error processing {file_name}: {str(e)}")
            continue

In [ ]:
union_ppi = pd.read_csv('processed-data/union_ppi.txt', sep='\t', header=None)
union_ppi.head()

In [ ]:
id_map = pd.read_csv('processed-data/mapped_ids_reviewed.csv', sep='\t')
id_map.head()

In [ ]:
folder_path = 'pathwaycommons-individual'
process_all_pathways(folder_path, union_ppi, id_map)

In [6]:
folder_path = 'training-samples'
files = os.listdir(folder_path)
num_files = len(files)
print(f"Total files: {num_files}")

Total files: 821
